In [13]:
import pickle
import pandas as pd
import numpy as np
import re
from tqdm import tqdm


def extract_date(text):
    # Helper function to extract date in the format dd/mm/yyyy
    match = re.search(r'(\d{2}/\d{2}/\d{4})', text)
    if match:
        return pd.to_datetime(match.group(1), dayfirst=True)
    return None

def extract_disembark_loc(text):
    # Helper function to extract the disembark location from text and clean it up
    match = re.search(r'débarqué[es]? à ([A-Za-zÀ-ÿ\'-]+(?: [A-Za-zÀ-ÿ\'-]+)*)', text, flags=re.IGNORECASE)
    if match:
        place_name = match.group(1)
        # Remove trailing non-place words like "le", "la", "du", etc.
        place_name = re.sub(r'\b(?:le|la|du|en|aux)\b.*$', '', place_name, flags=re.IGNORECASE).strip()
        return place_name
    return None

def split_remarks(remarks):
    # Split remarks based on 'rembarqué' keyword to handle each segment separately
    parts = re.split(r'(rembarqué[es]?)', remarks, flags=re.IGNORECASE)
    segments = [parts[0]]  # First part before 'rembarqué'
    
    for i in range(1, len(parts) - 1, 2):
        segments.append(parts[i] + parts[i + 1])  # Combine 'rembarqué' with the following text
    
    return segments

def process_rembarque(df):
    new_rows = []

    # Filter rows that contain the "rembarqué" keyword in the Remarks column
    rembarque_rows = df[df['Remarks'].str.contains('rembarqué', na=False, case=False)]

    for index, row in rembarque_rows.iterrows():
        segments = split_remarks(row['Remarks'])

        # Process the first segment (before the first rembarqué)
        first_segment = segments[0]
        first_embark_date = row['Emb_date']  # Use original embark date
        first_disembark_date = extract_date(re.search(r'débarqué[es]?.*', first_segment, flags=re.IGNORECASE).group()) if re.search(r'débarqué[es]?.*', first_segment, flags=re.IGNORECASE) else None
        first_disembark_loc = extract_disembark_loc(first_segment)  # Extract disembark location

        original_row = row.copy()
        original_row['Emb_date'] = first_embark_date  # Keep original embark date
        original_row['Disemb_date'] = first_disembark_date if first_disembark_date else row['Disemb_date']
        original_row['Disemb_loc'] = first_disembark_loc if first_disembark_loc else row['Disemb_loc']
        original_row['Remarks'] = first_segment.strip()
        new_rows.append(original_row)

        previous_disembark_date = first_disembark_date

        # Process subsequent segments starting with "rembarqué"
        for i, segment in enumerate(segments[1:], start=1):
            # Handle "rembarqué le dit jour et an"
            if 'rembarqué le dit jour et an' in segment.lower():
                embark_date = previous_disembark_date
            else:
                embark_date = extract_date(re.search(r'rembarqué[es]?.*', segment, flags=re.IGNORECASE).group()) if re.search(r'rembarqué[es]?.*', segment, flags=re.IGNORECASE) else None

            disembark_date = extract_date(re.search(r'débarqué[es]?.*', segment, flags=re.IGNORECASE).group()) if re.search(r'débarqué[es]?.*', segment, flags=re.IGNORECASE) else None
            disembark_loc = extract_disembark_loc(segment)  # Extract disembark location

            new_row = row.copy()
            new_row['Emb_date'] = embark_date
            new_row['Disemb_date'] = disembark_date if disembark_date else row['Disemb_date']
            
            # Only the last segment should keep the original Disemb_loc; others should extract it
            if i == len(segments) - 1:
                new_row['Disemb_loc'] = row['Disemb_loc']
            else:
                new_row['Disemb_loc'] = disembark_loc if disembark_loc else row['Disemb_loc']

            new_row['Remarks'] = segment.strip()
            new_rows.append(new_row)

            # Update the previous disembark date
            previous_disembark_date = disembark_date if disembark_date else previous_disembark_date

    # Append the processed rows back to the original dataframe
    non_rembarque_rows = df[~df.index.isin(rembarque_rows.index)]
    new_df = pd.concat([non_rembarque_rows, pd.DataFrame(new_rows)], ignore_index=True)

    # Sort the dataframe by Last Name and Embark Date
    new_df = new_df.sort_values(by=['Last Name', 'Emb_date'])

    return new_df

def replace_nan_array(val):
    if isinstance(val, np.ndarray):
        if val.size == 1 and np.isnan(val[0]):
            return np.nan
    return val

def match_or_nan(a, b):
    # Function to check if two values are equal or both are NaN
    return (a == b) | (pd.isna(a) & pd.isna(b))

def check_issues(row):
    issues = []
    if not pd.isna(row['Emb_date']) and not pd.Series(row['Emb_date']).str.match(r'^\d{2}/\d{2}/\d{4}$', na=False).bool():
        issues.append('emb_date')
    if pd.isna(row['Emb_date']):
        issues.append('emb_date')
    if not pd.isna(row['Disemb_date']) and not pd.Series(row['Disemb_date']).str.match(r'^\d{2}/\d{2}/\d{4}$', na=False).bool():
        issues.append('disemb_date')
    if pd.isna(row['Disemb_date']):
        issues.append('Disemb_date')
    if pd.isna(row['Emb_loc']):
        issues.append('emb_loc')
    if pd.isna(row['Disemb_loc']):
        issues.append('disemb_loc')
    return issues


def convert_to_ddmmyyyy(date_value):
    if pd.isna(date_value) or str(date_value).lower() in ['nan', '[nan]']:
        return np.nan
    try:
        # Parse the date and reformat to dd/mm/yyyy
        return pd.to_datetime(date_value).strftime('%d/%m/%Y')
    except (ValueError, TypeError):
        return date_value


def extract_details(text):
    # Patterns for embarkation
    embark_patterns = {
        'embarqué': {                                                                                           
            'date': r'\b(?:embarqué|rembarqué|embarquée|rembarquée|embarqués|rembarqués|embarquées|rembarquées) (?:(?!\b(?:débarqué|noyé(?:e|es|s)?|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées|resté)\b).)*(?:le|du) (\d{2}/\d{2}/\d{4})(?=\s|\n|$| ---|,)',
            'location': r'\b(?:embarqué|rembarqué|embarquée|rembarquée|embarqués|rembarqués|embarquées|rembarquées) (?:comme mousse à|en remplacement à|clandestinement au départ de|clandestinement à|furtivement à|furtivement au|au|à|en supplément à|en) ([\w\s\'-îÎ]+?)(?=\s+\b(?:pour la|départ|fin|mort(?:e|es|s)?|débarqué(?:e|es|s)?|déserté(?:e|es|s)?|passé(?:e|es|s)?|resté(?:e|es|s)?|tombé(?:e|es|s)?|repassé(?:e|es|s)?)\b|en remplacement|en supplément| (?:le|du) \d{2}/\d{2}/\d{4}|,| ---|\n|$| a servi à bord)'
        },
        'a fait la': {
            'date': r'remplacement (?:le|du) (\d{2}/\d{2}/\d{4})',
            'location': r'\b(?:a|à) fait la campagne de ([\w\s\'-îÎ]+?) à ([\w\s\'-îÎ]+)'
        },
        'supplément': {
            'date': r'\bsupplément (?:à|au) (?!\b(?:débarqué|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées)\b)[\w\s\'-îÎ]+? (?:du|le) (\d{2}/\d{2}/\d{4})(?=\s|\n|$| ---|,)',
            'location': r'\bsupplément (?:à|au) ([\w\s\'-îÎ]+?)(?=\s|\n|$|(?:du|le) \d{2}/\d{2}/\d{4}| ---|,)'
        },
        'complément': {
            'date': r'\bcomplément (?:à|au) (?!\b(?:débarqué|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées)\b)[\w\s\'-îÎ]+? (?:du|le) (\d{2}/\d{2}/\d{4})(?=\s|\n|$| ---|,)',
            'location': r'\bcomplément (?:à|au) ([\w\s\'-îÎ]+?)(?=\s|\n|$|(?:du|le) \d{2}/\d{2}/\d{4}| ---|,)'
        },
        'remplacement': {
            'date': r'\b(?:remplacement|remplaçement) (?:au|à|en)?(?!\b(?:débarqué|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées)\b)[\w\s\'-îÎ]*?(?:le|du) (\d{2}/\d{2}/\d{4})',
            'location': r'\b(?:remplacement|remplaçement) (?:en rade de|au|à|en)? ([\w\s\'-îÎ]+?)(?= débarqué(?:s|es|s)?| déserté(?:s|es|s)?| mort(?:s|es|s)?| passé(?:s|es|s)?| en| le| du|\n|,| ---|$)'
        },
        'embanremp': {
            'date': r'\bembarqué en remplacement (?:au|à)?(?!\b(?:débarqué|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées)\b)[\w\s\'-îÎ]*?(?:le|du) (\d{2}/\d{2}/\d{4})',
            'location': r'\bembarqué en remplacement (?:au|à)? ([\w\s\'-îÎ]+?)(?= débarqué(?:s|es|s)?| déserté(?:s|es|s)?| mort(?:s|es|s)?| passé(?:s|es|s)?| en| le| du|\n|,| ---|$)'
        },
        'trouvé': {
            'date': r'\btrouvé (?:(?!\b(?:quitté(?:e|es|s)?|matelot|débarqué|noyé(?:e|es|s)?|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées|resté)\b).)*\s*(?:le|du) (\d{2}/\d{2}/\d{4})(?=\s|\n|$| ---|,)',
            'location': r'\btrouvé(?:e|es|s)? (?:à bord au départ du cap de|à bord au départ du|à bord en mer|à bord en partant de|à bord en partant du|à bord en quittant le|à bord au moment du départ de|à bord au moment du départ|à bord du vaisseau après le départ de|à bord après le départ de|à bord après le départ|dans le vaisseau au départ de|dans le vaisseau|à bord du vaisseau|caché(?:e|es|s)? au départ de|caché(?:e|es|s)? à bord au départ de|caché(?:e|es|s)? à bord au départ|caché(?:e|es|s)? à bord|caché(?:e|es|s)? à|à bord au départ de|à bord à|à bord|au départ de|à|au) (?!\s*(?:le|du)\b)([\w\s\'-îÎ]+?)?(?=\s*(?:le|du) \d{2}/\d{2}/\d{4}|a servi|---|,|\n|$|\b(?:débarqué(?:e|es|s)?|déserté(?:e|es|s)?|mort(?:e|es|s)?|passé(?:e|es|s)?))'
        }, 
        'soldat': {
            'date': r'\bsoldat(?:e|es|s)? à bord le\s*(\d{2}/\d{2}/\d{4})(?:\s|,| ---|$)',
            'location': r''
        },
        'né': {
            'date': r'\bné(?:e|es|s)? (?:à bord|en mer) (?:le|du) (\d{2}/\d{2}/\d{4})',
            'location': r''
        }
    }

    # Patterns for disembarkation
    disembark_patterns = {
        'débarqué': {
            'date': r'\bdébarqué(?:e|es|s)? (?:au service du roi à |à l\'hôpital de |à |au |en échange à |furtivement à |malade après être tombé du haut mal à |malade à l\'hôpital (?:de|du) |malade à |malade et mort à l\'hôpital (?:de|du) |malade au |à |au |en )?[^\d\n,]*?(?:le|du) (\d{2}/\d{2}/\d{4})(?=(\s*---|\n|$|\s*passé(?:s|e|es)?|\s*échappé(?:e|s|es)?|\s*mort(?:e|s|es)?|\s*noyé(?:e|s|es)?))',
            'location': r'\bdébarqué(?:e|es|s)? (?:au service du roi à |à l\'hôpital de |en échange à |furtivement à |malade après être tombé du haut mal à |malade à l\'hôpital (?:de|du) |malade à |malade et mort à l\'hôpital (?:de|du) |malade au |à |au |en )?([\w\s\'-îÎ]+?)(?=( ---|---|\s+(?:le|du)\s+\d{2}/\d{2}/\d{4}|(?:le|du)\s+\d{2}/\d{2}/\d{4}|(?:le|du)\s+\?|\n|$|et n\'a pas|\s*passé(?:s|e|es)?|échappé(?:e|s|es)?| échappé(?:e|s|es)?|mort(?:e|s|es)?| mort(?:e|s|es)?|noyé(?:e|s|es)?| noyé(?:e|s|es)?))'
        },
        'déserté': {
            'date': r'\bdéserté(?:e|es|s)?(?: à [\w\s\'\-îÎ]+| sur le vaisseau de côte le [\w\s\'\-îÎ]+| en [\w\s\'\-îÎ]+| au départ à [\w\s\'\-îÎ]+| au [\w\s\'\-îÎ]+)? (?:le|du) (\d{2}/\d{2}/\d{4})',
            'location': r'\bdéserté(?:e|es|s)? (?:en venant au|à l\'hôpital du|à|en|au départ de|au départ à|au) ([\w\s\'-îÎ]+?)(?: le|\n|,| ---|$| au départ)'
        },
        'mort en mer': {
            'date': r'\b(?:mort(?:e|es|s)?|noyé(?:e|es|s)?) (?:en mer |noyé en [\w\s\'-îÎ]+ |du [\w\s\'-îÎ]+ en [\w\s\'-îÎ]+ |à l\'hôpital de [\w\s\'-îÎ]+ |à la ration |à [\w\s\'-îÎ]+ |tombé à la mer et mort noyé |dans le Gange |au [\w\s\'-îÎ]+ |en [\w\s\'-îÎ]+ )?(?:le|le )(\d{2}/\d{2}/\d{4})',
            'location': r'\b(?:mort(?:e|es|s)?|noyé(?:e|es|s)?) (?:en mer|à l\'hôpital (?:du|de)|à la ration|à|au|en) ([\w\s\'-îÎ]+?)(?= le \d{2}/\d{2}/\d{4}| le\d{2}/\d{2}/\d{4}| ---| \n)'
        },
        'a fait la': {
            'date': r'levé(?:e|es|s)? du (\d{2}/\d{2}/\d{4})',
            'location': r'\b(?:a|à) fait la campagne de [\w\s\'-îÎ]+? à ([\w\s\'-îÎ]+?)(?=\s*(?:du|le) \d{2}/\d{2}/\d{4}|\s*---|,|en|\n|$)'
        },
        'passé': {
            'date': r'\bpassé(?:e|es|s)? sur la [\w\s\'-îÎ]+ en rade de [\w\s\'-îÎ]+ le (\d{2}/\d{2}/\d{4})',
            'location': r'\bpassé(?:e|es|s)? sur la [\w\s\'-îÎ]+ en rade de ([\w\s\'-îÎ]+)(?=\s+(?:le\s+\d{2}/\d{2}/\d{4}|,| ---|\n|$))'
        },
        'resté': {
            'date': r'\bresté(?:e|es|s)?(?:\s+(?:malade à terre à|à terre à|à terre|à l\'hôpital à|à l\'hôpital de|malade au départ de|malade à l\'hôpital de|à terre au départ de|à terre malade au départ de|à terre malade à|à|en|au départ de|malade à|malade au|malade)?(?: [\w\s\'-îÎ]+?)?)? le\s*(\d{2}/\d{2}/\d{4})(?:\b.*?| rejoint|\n|,| ---|$)',
            'location': r'\bresté(?:e|es|s)? (?:malade à terre à|à terre à|à terre|à l\'hôpital à|à l\'hôpital de|malade au départ de|malade à l\'hôpital de|à terre au départ de|à terre malade au départ de|à terre malade à|à|en|au départ de|malade à|malade au|malade) ([\w\s\'-îÎ]+?)(?: en supplément à| le| rejoint|\n|,| ---|$| au départ)'
        }
    }

    # Initialize results
    embark_location = pd.NA
    embark_date = pd.NA
    disembark_location = pd.NA
    disembark_date = pd.NA

    # Initialize match variables
    embark_date_match = None
    embark_location_match = None
    disembark_date_match = None
    disembark_location_match = None

    # Determine which embark patterns to use
    if 'embarqué en remplacement' in text.lower():
        embark_date_match = re.search(embark_patterns['embanremp']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['embanremp']['location'], text, flags=re.IGNORECASE)
    elif 'embarqué' in text.lower() or 'rembarqué' in text.lower():
        embark_date_match = re.search(embark_patterns['embarqué']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['embarqué']['location'], text, flags=re.IGNORECASE)
    elif 'fait la campagne' in text.lower():
        embark_date_match = re.search(embark_patterns['a fait la']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['a fait la']['location'], text, flags=re.IGNORECASE)
    elif 'supplément' in text.lower():
        embark_date_match = re.search(embark_patterns['supplément']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['supplément']['location'], text, flags=re.IGNORECASE)
    elif 'remplacement' in text.lower() or 'remplaçement' in text.lower():
        embark_date_match = re.search(embark_patterns['remplacement']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['remplacement']['location'], text, flags=re.IGNORECASE)
    elif 'complément' in text.lower():
        embark_date_match = re.search(embark_patterns['complément']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['complément']['location'], text, flags=re.IGNORECASE)        
    elif 'trouvé' in text.lower():
        embark_date_match = re.search(embark_patterns['trouvé']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['trouvé']['location'], text, flags=re.IGNORECASE)        
    elif 'soldat à bord le' in text.lower():
        embark_date_match = re.search(embark_patterns['soldat']['date'], text, flags=re.IGNORECASE)
    elif re.search(r'\bné(?:e|es|s)?\b', text.lower()):
        embark_date_match = re.search(embark_patterns['né']['date'], text, flags=re.IGNORECASE)

    # Extract embark information
    if embark_date_match:
        embark_date = embark_date_match.group(1)
    if embark_location_match:
        embark_location = embark_location_match.group(1)

    # Determine which disembark patterns to use
    if 'débarqué' in text.lower():
        disembark_date_match = re.search(disembark_patterns['débarqué']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['débarqué']['location'], text, flags=re.IGNORECASE)
    elif 'déserté' in text.lower():
        disembark_date_match = re.search(disembark_patterns['déserté']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['déserté']['location'], text, flags=re.IGNORECASE)
    elif 'fait la campagne' in text.lower():
        disembark_date_match = re.search(disembark_patterns['a fait la']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['a fait la']['location'], text, flags=re.IGNORECASE)
    elif 'resté' in text.lower():
        disembark_date_match = re.search(disembark_patterns['resté']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['resté']['location'], text, flags=re.IGNORECASE)
    elif 'passé' in text.lower():
        disembark_date_match = re.search(disembark_patterns['passé']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['passé']['location'], text, flags=re.IGNORECASE)
    elif 'mort' in text.lower() or 'noyé' in text.lower():
        disembark_date_match = re.search(disembark_patterns['mort en mer']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['mort en mer']['location'], text, flags=re.IGNORECASE)

    # Extract disembark information
    if disembark_date_match:
        disembark_date = disembark_date_match.group(1)
    if disembark_location_match:
        disembark_location = disembark_location_match.group(1)
#         if len(disembark_location_match.groups()) > 1 and disembark_location_match.group(2):
#             disembark_location += f", {disembark_location_match.group(2)}"
    if 'mort à bord' in text.lower() or 'morte à bord' in text.lower() or 'morts à bord' in text.lower() or 'mortes à bord' in text.lower():
        disembark_location = 'on board'
    elif 'mort en mer' in text.lower() or 'morte en mer' in text.lower() or 'morts en mer' in text.lower() or 'mortes en mer' in text.lower():
        disembark_location = 'on board'
    if 'née en mer' in text.lower() or 'né en mer' in text.lower() or 'nées en mer' in text.lower() or 'nés en mer' in text.lower():
        embark_location = 'on board'
    elif 'née à bord' in text.lower() or 'né à bord' in text.lower() or 'nées à bord' in text.lower() or 'nés à bord' in text.lower():
        embark_location = 'on board'
    
            

    return embark_location, embark_date, disembark_location, disembark_date

def fill_emb_loc_for_rembarque(df):
    # Sort the dataframe by Last Name, Emb_date, and Disemb_date
    df_sorted = df.sort_values(by=['Last Name', 'Emb_date', 'Disemb_date']).reset_index(drop=True)
    
    # Iterate through the sorted dataframe
    for i in range(1, len(df_sorted)):
        current_row = df_sorted.iloc[i]
        previous_row = df_sorted.iloc[i - 1]
        
        # Check if Remarks is a string and contains the keyword "rembarqué"
        if pd.isna(current_row['Emb_loc']) and isinstance(current_row['Remarks'], str) and 'rembarqué' in current_row['Remarks'].lower():
            # Set the current row's Emb_loc to the previous row's Disemb_loc
            df_sorted.at[i, 'Emb_loc'] = previous_row['Disemb_loc']
    
    return df_sorted

In [14]:
with open('errors2.pickle', 'rb') as f:
    errors = pickle.load(f)

df_s = pd.read_csv('combined_filled.csv').drop('Unnamed: 0', axis=1)

In [15]:
combined = []
new_errors = []
for code in tqdm(errors):
    try:
        if isinstance(code, str) and code != 'nan':
            df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
            voyage_code = code
            df = df[df['voyage_code']==voyage_code]
            steps_l = list(df.dropna(subset='steps').drop_duplicates(subset='steps')['steps'])
            if len(steps_l) > 0:
                steps = steps_l[0]
                stops = []
                if isinstance(steps, str):
                    clean = steps.replace(",", "-").replace("→", "-")
                    temp = re.split(r'(?<=\w)-(?=\s)|(?<=\s)-(?=\w)|(?<=\s)-(?=\s)', clean)
                    for j in temp:
                        stops.append(j.replace("?", "").strip())
                sl = stops[0]
                ll = stops[-1]
            else:
                sl_l = list(df.dropna(subset='start_loc').drop_duplicates(subset='start_loc')['start_loc'])
                ll_l = list(df.dropna(subset='last_loc').drop_duplicates(subset='last_loc')['last_loc'])
                if len(sl_l) > 0:
                    sl = sl_l[0]
                else:
                    sl = '?'
                if len(ll_l) > 0:
                    ll = ll_l[0]
                else:
                    ll = '?'
            dvb_l = list(df.dropna(subset='date_voy_begin').drop_duplicates(subset='date_voy_begin')['date_voy_begin'])
            dve_l = list(df.dropna(subset='date_voy_end').drop_duplicates(subset='date_voy_end')['date_voy_end'])
            if len(dvb_l) > 0:
                dvb = dvb_l[0]
            else:
                dvb = '11/11/1111'
            if len(dve_l) > 0:
                dve = dve_l[0]
            else:
                dve = '11/11/1111'
            df = df[['n', 'voyage_code', 'last_name', 'first_name', 'f_n', 'remarks', 'age', 'wage',
                   'emb_date', 'disemb_date', 'disemb_loc']].drop_duplicates()
            df.rename(columns={'last_name': 'Last Name', 'first_name': 'First Name', 'f_n': 'Function', 'remarks': 'Remarks', 'age': 'Age', 'wage': 'Wage',
                   'emb_date': 'Emb_date', 'disemb_date': 'Disemb_date', 'disemb_loc': 'Disemb_loc', 'voyage_code': 'Voyage Code'}, inplace=True)
            for col in df.select_dtypes(include=['object']):
                df[col] = df[col].str.strip()
            df.replace('[nan]', np.nan, inplace=True)
            df.replace('<Disemb_Loc>', np.nan, inplace=True)

            df = process_rembarque(df)

            df['Emb_loc'] = None
            for index, row in df.iterrows():
                remark = re.sub(r'\s+', ' ', str(row['Remarks']).replace('\n', ' '))

                emb_date = row['Emb_date']
                disemb_date = row['Disemb_date']
                disemb_loc = row['Disemb_loc']
                embark_location, embark_date, disembark_location, disembark_date = extract_details(remark)

                df.at[index, 'Emb_loc'] = embark_location
                if pd.notna(disembark_date):
                    df.at[index, 'Disemb_date'] = disembark_date
                if pd.notna(embark_date):
                    df.at[index, 'Emb_date'] = embark_date
                if pd.notna(disembark_location):
                    df.at[index, 'Disemb_loc'] = disembark_location

            df = fill_emb_loc_for_rembarque(df)

            des = ll
            arm = sl
            for index, row in df.iterrows():
                if pd.notna(row['Remarks']):
                    if isinstance(row['Disemb_loc'], str) and ('désarmement' in row['Disemb_loc'].lower() or row['Disemb_loc'].lower()=='désarmement'):
                        df.at[index, 'Disemb_loc'] = des

                        if pd.isna(row['Disemb_date']):
                            df.at[index, 'Disemb_date'] = dve

                    if isinstance(row['Emb_loc'], str) and 'armement' in row['Emb_loc'].lower():
                        df.at[index, 'Emb_loc'] = arm

                        if pd.isna(row['Emb_date']):
                            df.at[index, 'Emb_date'] = dvb

                    if 'a fait la campagne' in row['Remarks'].lower():
                        if pd.isna(row['Emb_date']):
                            df.at[index, 'Emb_date'] = dvb
                        if pd.isna(row['Disemb_date']):
                            df.at[index, 'Disemb_date'] = dve
                        if pd.isna(row['Emb_loc']):
                            df.at[index, 'Emb_loc'] = arm
                        if pd.isna(row['Disemb_loc']):
                            df.at[index, 'Disemb_loc'] = des

            df['Emb_date'] = df['Emb_date'].apply(convert_to_ddmmyyyy)
            df['Disemb_date'] = df['Disemb_date'].apply(convert_to_ddmmyyyy)
            df['Emb_date'] = df['Emb_date'].apply(lambda x: x if isinstance(x, str) else np.nan)
            df['Disemb_date'] = df['Disemb_date'].apply(lambda x: x if isinstance(x, str) else np.nan)

            df['emb_class'], df['disemb_class'] = None, None
            for index, row in df.iterrows():
                if not isinstance(row['Remarks'], str):
                    continue
                if 'clandestin' in row['Remarks'] or 'caché' in row['Remarks'] or 'furtivement' in row['Remarks']:
                    df.at[index, 'emb_class'] = 304
                elif 'rembarqué' in row['Remarks']:
                    df.at[index, 'emb_class'] = 306
                elif 'fait la campagne' in row['Remarks'] or 'armement' in row['Remarks'] or 'embarqué' in row['Remarks']:
                    if pd.notna(row['Emb_loc']):
                        if row['Emb_loc'] == arm:
                            df.at[index, 'emb_class'] = 301
                        else:
                            df.at[index, 'emb_class'] = 302
                elif 'remplacement' in row['Remarks'] or 'supplément' in row['Remarks']:
                    df.at[index, 'emb_class'] = 302
                elif 'resté' in row['Remarks']:
                    df.at[index, 'emb_class'] = 308
                elif 'renversement' in row['Remarks'] or 'vient' in row['Remarks']:
                    df.at[index, 'emb_class'] = 303
                elif 'né' in row['Remarks']:
                    df.at[index, 'emb_class'] = 305
                else:
                    df.at[index, 'emb_class'] = 309

                if 'mort' in row['Remarks'] or 'noyé' in row['Remarks']:
                    df.at[index, 'disemb_class'] = 305
                elif 'fait la campagne' in row['Remarks']:
                    df.at[index, 'disemb_class'] = 301
                elif 'resté malade' in row['Remarks'] or 'resté à l\'hôpital' in row['Remarks'] or 'restée malade' in row['Remarks'] or 'restée à l\'hôpital' in row['Remarks']:
                    df.at[index, 'disemb_class'] = 306
                elif 'débarqué' in row['Remarks']:
                    if pd.notna(row['Disemb_loc']):
                        if row['Disemb_loc'] == des:
                            df.at[index, 'disemb_class'] = 301
                        else:
                            df.at[index, 'disemb_class'] = 302
                elif 'passé' in row['Remarks']:
                    df.at[index, 'disemb_class'] = 303
                elif 'échappé' in row['Remarks'] or 'fugitif' in row['Remarks'] or 'déserté' in row['Remarks']:
                    df.at[index, 'disemb_class'] = 304
                elif 'prise par le' in row['Remarks']:
                    df.at[index, 'disemb_class'] = 307
                elif 'resté' in row['Remarks']:
                    df.at[index, 'disemb_class'] = 308
                else:
                    df.at[index, 'disemb_class'] = 309

            reembark_rows = df[(df['Remarks'].str.contains('rembarqué', na=False, case=False)) & (df['Emb_loc'].isna())]

            for index, row in reembark_rows.iterrows():

                matching_rows = df[
                    match_or_nan(df['Last Name'], row['Last Name']) &
                    match_or_nan(df['First Name'], row['First Name']) &
                    match_or_nan(df['Function'], row['Function']) &
                    (df['Last Name']!='Anonymous')
                ]

                sorted_group = matching_rows.sort_values(by=['Emb_date', 'Disemb_date'])

                previous_disemb_loc = None
                for i, (sorted_index, sorted_row) in enumerate(sorted_group.iterrows()):
                    if sorted_index == index:
                        if previous_disemb_loc:
                            df.at[sorted_index, 'Emb_loc'] = previous_disemb_loc

                    previous_disemb_loc = sorted_row['Disemb_loc']

            for _, group in df.groupby(['Last Name', 'First Name', 'Function'], dropna=False):
                sorted_group = group.sort_values(by=['Emb_date', 'Disemb_date'])
                for i, (index, row) in enumerate(sorted_group.iterrows()):
                    if pd.isna(row['Disemb_loc']) and i < len(sorted_group) - 1:
                        if row['Last Name'] != 'Anonymous':
                            next_row = sorted_group.iloc[i + 1]
                            df.at[index, 'Disemb_loc'] = next_row['Emb_loc']

            df['Emb_date'] = df['Emb_date'].apply(convert_to_ddmmyyyy)
            df['Disemb_date'] = df['Disemb_date'].apply(convert_to_ddmmyyyy)
            df['Emb_date'] = df['Emb_date'].apply(lambda x: x if isinstance(x, str) else np.nan)
            df['Disemb_date'] = df['Disemb_date'].apply(lambda x: x if isinstance(x, str) else np.nan)
            df['Emb_loc'] = df['Emb_loc'].apply(lambda x: x if isinstance(x, str) else np.nan)
            df['Disemb_loc'] = df['Disemb_loc'].apply(lambda x: x if isinstance(x, str) else np.nan)
            df.replace('[nan]', np.nan, inplace=True)

            for index, row in df.iterrows():
                if isinstance(row['Remarks'], str) and ('mort' in row['Remarks'] or 'noyé' in row['Remarks']):
                    if pd.notna(row['Disemb_date']):
                        df.at[index, 'Disemb_loc'] = 'mort le '+row['Disemb_date']
                        df.at[index, 'Disemb_date'] = np.nan
                    else:
                        df.at[index, 'Disemb_loc'] = 'mort en mer'
                if isinstance(row['Remarks'], str) and re.search(r'\bné(?:e|s|es)?\b', row['Remarks']):
                    if pd.notna(row['Emb_date']):
                        df.at[index, 'Emb_loc'] = 'né le '+row['Emb_date']
                        df.at[index, 'Emb_date'] = np.nan
                    else:
                        df.at[index, 'Emb_loc'] = 'né en mer'
                if isinstance(row['Emb_loc'], str) and 'départ' in row['Emb_loc']:
                    df.at[index, 'Emb_loc'] = np.nan
                if isinstance(row['Disemb_loc'], str) and 'départ' in row['Disemb_loc']:
                    df.at[index, 'Disemb_loc'] = np.nan
                if isinstance(row['Emb_loc'], str) and ('a servi' in row['Emb_loc'] or 'le lendemain' in row['Emb_loc'] or 'vaisseau' in row['Emb_loc']):
                    df.at[index, 'Emb_loc'] = np.nan
            df['Voyage Code'] = str(code)

            combined.append(df)
    except:
        print('************************************************', code)
        new_errors.append(code)

  0%|          | 0/177 [00:00<?, ?it/s]<ipython-input-15-d68b4828f690>:6: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
  1%|          | 2/177 [00:06<10:09,  3.48s/it]<ipython-input-15-d68b4828f690>:6: DtypeWarning: Columns (30,43,44,49) have mixed t

  7%|▋         | 13/177 [21:31<4:57:21, 108.79s/it]<ipython-input-15-d68b4828f690>:6: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
  8%|▊         | 14/177 [26:39<7:38:51, 168.91s/it]<ipython-input-15-d68b4828f690>:6: DtypeWarning: Columns (30,43,44,

<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
 14%|█▍        | 25/177 [38:02<2:37:33, 62.19s/it]<ipython-input-15-d68b4828f690>:6: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
 15%|█▍        | 26/177 [38:10<1:55:40, 45.96s/it]

************************************************ AIG01


<ipython-input-15-d68b4828f690>:6: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
 15%|█▌        | 27/177 [43:16<5:09:34, 123.83s/it]<ipython-input-15-d68b4828f690>:6: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on impor

...

<ipython-input-15-d68b4828f690>:6: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
 74%|███████▍  | 131/177 [2:14:08<05:11,  6.77s/it]<ipython-input-15-d68b4828f690>:6: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on impor

 79%|███████▉  | 140/177 [2:15:09<04:10,  6.76s/it]<ipython-input-15-d68b4828f690>:6: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
 80%|███████▉  | 141/177 [2:15:15<04:03,  6.77s/it]<ipython-input-15-d68b4828f690>:6: DtypeWarning: Columns (30,43,44,

<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
 86%|████████▌ | 152/177 [2:16:29<02:45,  6.62s/it]<ipython-input-15-d68b4828f690>:6: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default)

 91%|█████████ | 161/177 [2:17:28<01:44,  6.53s/it]<ipython-input-15-d68b4828f690>:6: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
 92%|█████████▏| 162/177 [2:17:35<01:37,  6.53s/it]<ipython-input-15-d68b4828f690>:6: DtypeWarning: Columns (30,43,44,

 98%|█████████▊| 173/177 [2:18:47<00:27,  6.83s/it]<ipython-input-15-d68b4828f690>:6: DtypeWarning: Columns (30,43,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MOTHER_TABLE.csv').drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
<ipython-input-13-bd5942a52f9d>:127: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')
 98%|█████████▊| 174/177 [2:18:54<00:20,  6.74s/it]<ipython-input-15-d68b4828f690>:6: DtypeWarning: Columns (30,43,44,

In [16]:
new_errors

['AIG01', 'DPR01', 'ACH05', 'SYD01']

In [17]:
with open('errors3.pickle', 'wb') as f:
    pickle.dump(new_errors, f)

In [18]:
df

,n,Voyage Code,Last Name,First Name,Function,Remarks,Age,Wage,Emb_date,Disemb_date,Disemb_loc,Emb_loc,emb_class,disemb_class
0,5.0,OIS02,BRIAND,Golvin,NaN,embarqué à l'armement \n débarqué à ? --- de g...,NaN,16,07/06/1739,NaN,?,Lorient,301,302
1,12.0,OIS02,CASTEAU,Jean Baptiste,NaN,embarqué à l'armement \n débarqué à ?,NaN,18,07/06/1739,NaN,?,Lorient,301,302
2,13.0,OIS02,CORNET,Jacques,NaN,embarqué à l'armement \n débarqué à ?,20.0,14,07/06/1739,NaN,?,Lorient,301,302
3,15.0,OIS02,CORVOIZIER DESPORTES,Pierre René,NaN,embarqué à l'armement \n débarqué à ?,16.0,6,07/06/1739,NaN,?,Lorient,301,302
4,8.0,OIS02,DERVAL,Guillaume,NaN,embarqué à l'armement \n débarqué à ? --- de g...,NaN,13,07/06/1739,NaN,mort en mer,Lorient,301,305
5,10.0,OIS02,FORLEAU,Henri,NaN,embarqué à l'armement \n débarqué à ? --- de g...,NaN,12,07/06/1739,NaN,?,Lorient,301,302
6,9.0,OIS02,GESTIN,François,NaN,embarqué à l'armement \n débarqué à ? --- de g...,NaN,13,07/06/1739,NaN,?,Lorient,301,302
7,3.0,OIS02,GOURDUN,René,NaN,embarqué à l'armement \n renversé sur le Saint...,NaN,25,06/08/1739,01/10/1739,NaN,Lorient,301,304
8,7.0,OIS02,JULOU,Julien Joseph,NaN,embarqué à l'armement \n débarqué à ? --- de g...,NaN,13,07/06/1739,NaN,?,Lorient,301,302
9,11.0,OIS02,LE PORT,Bertrand,NaN,embarqué à l'armement \n débarqué à ? --- d'au...,25.0,15,07/06/1739,NaN,?,Lorient,301,302


In [19]:
combined_filled = pd.concat(combined)
combined_filled

,n,Voyage Code,Last Name,First Name,Function,Remarks,Age,Wage,Emb_date,Disemb_date,Disemb_loc,Emb_loc,emb_class,disemb_class
0,14.0,REN11,ALOTE,Jean,NaN,embarqué à l'armement \n débarqué à ? --- doit...,NaN,15,15/12/1790,NaN,?,555,301,302
1,12.0,REN11,BOUVIER,Jacques,NaN,embarqué à l'armement \n débarqué à ? --- doit...,NaN,15,15/12/1790,NaN,?,555,301,302
2,16.0,REN11,DEFFANNO,Jean,NaN,embarqué à l'armement \n débarqué à ? --- doit...,NaN,15,15/12/1790,NaN,?,555,301,302
3,9.0,REN11,FANDES,Joseph,NaN,embarqué à l'armement \n débarqué à ? --- doit...,NaN,15,15/12/1790,NaN,?,555,301,302
4,3.0,REN11,GARDNER,Gideon,NaN,embarqué à l'armement \n débarqué à ? --- doit...,NaN,50,15/12/1790,NaN,?,555,301,302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,16.0,OIS02,POIRIER,Pierre,233.0,complément à l'armement le 16/06/1739 \n débar...,27.0,17,16/06/1739,NaN,?,Lorient,301,302
12,1.0,OIS02,PROTEAU,Pierre,NaN,embarqué à l'armement \n débarqué à ? --- reve...,NaN,70,07/06/1739,NaN,?,Lorient,301,302
13,6.0,OIS02,PROUDEC,Yves,NaN,embarqué à l'armement \n débarqué à ? --- de g...,23.0,14,07/06/1739,NaN,?,Lorient,301,302
14,2.0,OIS02,RIVALE,Jacques,NaN,embarqué à l'armement \n débarqué à ?,NaN,30,07/06/1739,NaN,?,Lorient,301,302


In [20]:
combined_filled.to_csv('combined_filled2.csv')